# this is a Sudoku-Solver from WWW (see below)
Adjusted in two ways:
### 1: if a puzzle has more than ONE solution (so not really a puzzle)
the number of tries can be set (default is maximal 2)
### 2: default this SudokuSolver tries to set the diagonals too to 1..9

# www
 https://raw.githubusercontent.com/ppmx/sudoku-solver/master/sudoku-z3.py
 
 https://github.com/ppmx/sudoku-solver/blob/master/sudoku-z3.py
 

In [1]:
#some global initializations:

from z3 import *

def show_obj(obj):
    for el in dir(obj):
        if el.startswith('_'):
            continue
        print(el)

#ONE solution with diags 
default_start = "..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7.."

#next default_start with TWO solution (diagonal too)
default_diagonal_two_solutions ='..1........6.4........51.93..4.......92...57.......9..45.79..........2........7..'

def two_concat(A, B):   #name cross changed ;-)
    return [(a + b) for a in A for b in B]

#diagonal Sudoku uit **5** in Sudoku Lente (van Heike)
# "..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7.."

givenElements = []

In [2]:
class SudokuPKHG:
    """
    An adjusted Sudoku-Solver from
    https://github.com/ppmx/sudoku-solver/blob/master/sudoku-z3.py
    sudoku-z3.py could run in a Terminal or python-notebook

    THIs class has a second parameter 'diagonalen' as default
    to force 'both' diagonals to be distinct too!
    
    you may use '' or 'R' or 'L' for three different approaches:
    diagonalen = '' for a normal Sudoku
    diagonalen = 'L' for a Sudoku with Left to right diagonal 1..9
    diagonalen = 'R' for a Sudoku with Right to left diagonal 1..9
    
    """
    def __init__(self, inputstr=default_start, diagonalen = 'LR'):
        global givenElements
        self.print_Sudoku(inputstr)
        self.puzzle = inputstr
        self.diagonalen = diagonalen
        #self.sudoku_dict = self.parse_grid(self.puzzle)
        self.elements = self.parse_grid()
        self.BaseSizeInfo = []
        self.ins = []
        self.givenNams = []
        for nam, val  in self.elements.items():
            if val == '.':
                continue
            self.givenNams.append(nam)
            givenElements.append([nam,int(val)])
            sme = Int(nam) == int(val)
            self.BaseSizeInfo.append(sme)
            #print('dbg L14 sme =  BaseSizeInfo[-1] = {}'.format(sme))
            #print('dbg L13 givenElements {}'.format(givenElements))
        nam81 = two_concat("ABCDEFGHI", "123456789")
        self.symbs = {e: Int(e) for e in nam81 } #a dict of 'A1': A1, ...
        #print('DBG L20 symbols = {}'.format(self.symbs))
      
        # assure that every cell holds a value of [1,9]
      
        ### hier invoegen
        self.Anames = {}
        allnames = two_concat('ABCDEFGHI','123456789')
        for i, el in enumerate(allnames):
            #print('L34 el {}'.format(el))
            tmp = default_start[i]
            if tmp == '.':
                self.Anames[el] = Int(el)
            else:
                tmp = int(default_start[i])
                #print(tmp)
                self.Anames[el] = Int(el) #??? mag niet?Int(tmp) ==  tmp
        self.cells_c = []
        for el in allnames:
            self.cells_c.append(And(Int(el) >= 1, Int(el) <= 9)) #??? self.Anames[el])
            
            # or to try yourself ;-)
            #self.cells_c.append(Or([Int(el) == i for i in range(1,10)])) #??? self.Anames[el])

        # assure that every row covers every value: 
        self.rows_c = []
        for row in "ABCDEFGHI":
            #print('\n DBG')
            #self.rows_c.append(Distinct([Int(row + col) for col in "123456789"]))
            self.rows_c.append(Distinct([Int(row + col) for col in "123456789"]))
        
        # assure that every column covers every value:
        self.cols_c = []
        for col in "123456789":
            self.cols_c.append( Distinct([Int(row + col) for row in "ABCDEFGHI"]))
        self.nxn_c = []
        
        # assure that every block covers every value:
        for i in range(3):
            for j in range(3):
                el = [Int("ABCDEFGHI"[m + i * 3] + "123456789"[n + j * 3]) \
                                           for m in range(3) for n in range(3)]
                self.nxn_c.append(Distinct(el))
                #self.nxn_c.append(Sum(el) == 45)
        
        ## if diagonals must be 1 .. 9 too:
        self.diag_LR_c = []
        self.diag_RL_c = []
        if type(self.diagonalen) == type(''):
            print('\n_________DBG L75______\n string {}\n'.format(self.diagonalen)) 
            if 'L' in self.diagonalen:
                LR = [Int(el + str(i + 1)) for i, el in enumerate('ABCDEFGHI')] 
                print('DBG LR {}'.format(LR))
                self.diag_LR_c = [Distinct([el for el in LR])]
            if 'R' in self.diagonalen:
                RL = [Int(el + str( 9 - i )) for i, el in enumerate('ABCDEFGHI')]
                print('DBG RL {}'.format(RL))
                self.diag_RL_c = [Distinct([el for el in RL])]
   
    def print_Sudoku(self, all81):
        """
    print a Sudoku (a list of  81 given elements, a '?')
    Example all81
    all81 ='..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7..'
    OR
    all81 = list(all81) for conviniance
        """
        all81_str =  ''
        if type(all81) == type([]):
            all81 = ''.join(all81)
        all81_str = all81 # ''.join(all81) #''.join(all81) #conversion to ONE 81 long string
        for i in '123456789':
            all81_str = all81_str.replace(i, ' ' + i + ' ')
        all81_str = all81_str.replace('.',' . ')
          
        aLL = [  all81_str[i * 27: i * 27 + 27] for i in range(9)]
        #plit into 9 lines
    
        result = []
          
        bo = '-'*10 + '.' +'-'*9 + '.' + '-'*10
        for i in range(9):
            if i % 3 == 0:
                print(bo)
            row = aLL[i]
        #print(row)
            row1 = [ row[i * 9: i * 9 + 9]  for i in range(3)]
            row1nn = '|' + '|'.join(row1) + '|'
            print(row1nn)
        print(bo)
        return 'done'


    def parse_grid(self):
        """
        from sudoku-z3.py see links
        
        A1 A2 A3 | A4 A5 A6 | A7 A8 A9
        B1 B2 B3 | B4 B5 B6 | B7 B8 B9
        C1 C2 C3 | C4 C5 C6 | C7 C8 C9
        –––––––––+––––––––––+–––––––––
        D1 D2 D3 | D4 D5 D6 | D7 D8 D9
        E1 E2 E3 | E4 E5 E6 | E7 E8 E9
        F1 F2 F3 | F4 F5 F6 | F7 F8 F9
        –––––––––+––––––––––+–––––––––
        G1 G2 G3 | G4 G5 G6 | G7 G8 G9
        H1 H2 H3 | H4 H5 H6 | H7 H8 H9
        I1 I2 I3 | I4 I5 I6 | I7 I8 I9

        puzzle = 'A1A2A3A4...' and every element holds a value of '123456789.'
        where the dot represents an empty cell.
        """
        puzzle = self.puzzle
        s = dict()

        if any(c not in "123456789." for c in puzzle) or len(puzzle) != 81:
            raise Exception("got invalid puzzle format")

        elements = two_concat("ABCDEFGHI", "123456789")
        s = {e: v for e,v in zip(elements, puzzle)}
        return s
    
    def run_z3(self, cmax=2):
        allnames = two_concat('ABCDEFGHI','123456789')
        self.countmax = cmax
        self.sol = Solver()
        self.sol.add(self.BaseSizeInfo)
        self.sol.add(self.nxn_c + self.cols_c + self.rows_c + self.cells_c)
    
        if type(self.diagonalen) == type(''):
            if 'L' in self.diagonalen:
                LR = [Int(el + str(i + 1)) for i, el in enumerate('ABCDEFGHI')]           
                self.diag_LR_c = [Distinct([el for el in LR])]
            if 'R' in self.diagonalen:
                RL = [Int(el + str( 9 - i )) for i, el in enumerate('ABCDEFGHI')]
                self.diag_RL_c = [Distinct([el for el in RL])]
    
        self.sol.add(self.diag_LR_c) #digagonal from Left to Right
        self.sol.add(self.diag_RL_c)
        self.count = 0
        countmax = self.countmax #if there are more than on solution
        self.snu = []
        #self.vals1 = []
        #self.vals2 = []
        solNames = [ Int(i + j) for i in 'ABCDEFGHI' for j in '123456789']
        
        while self.sol.check() == sat and self.count < countmax:
            self.count += 1
            print('count = ', self.count) 
            m = self.sol.model()
            self.snu = [m.evaluate(el) for el in solNames] #contains 81 name 
            r = [[self.snu[i + j * 9] for i in range(9)] for j in range(9)]
            for rij in r:
                print(rij)
            vals =[ Int(el) == m.evaluate(Int(el)) for el in allnames]
            
            #TO FORCE LOOKING FOR AN OTHER SOLUTION
            
            self.sol.add(Not(And(vals)))
            print("\n=============\n")

        #while ends here
        
        if self.count == 0:
            print('unsat')
        elif self.count == 1:
            print('solution found')
        else:
            print('count = {} '.format(self.count), '\n but more solutions possible!')
  



In [3]:
print(default_start, default_diagonal_two_solutions)

..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7.. ..1........6.4........51.93..4.......92...57.......9..45.79..........2........7..


In [4]:
ST = SudokuPKHG(default_start, diagonalen='RL' ) 
#"931........6.4........51.93..4.......92...57.......9..45.79........8.2........7.."

#,diagonalen ='RL')

----------.---------.----------
| .  .  1 | .  .  . | .  .  . |
| .  .  6 | .  4  . | .  .  . |
| .  .  . | .  5  1 | .  9  3 |
----------.---------.----------
| .  .  4 | .  .  . | .  .  . |
| .  9  2 | .  .  . | 5  7  . |
| .  .  . | .  .  . | 9  .  . |
----------.---------.----------
| 4  5  . | 7  9  . | .  .  . |
| .  .  . | .  8  . | 2  .  . |
| .  .  . | .  .  . | 7  .  . |
----------.---------.----------

_________DBG L75______
 string RL

DBG LR [A1, B2, C3, D4, E5, F6, G7, H8, I9]
DBG RL [A9, B8, C7, D6, E5, F4, G3, H2, I1]


In [5]:
help(ST)

Help on SudokuPKHG in module __main__ object:

class SudokuPKHG(builtins.object)
 |  SudokuPKHG(inputstr='..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7..', diagonalen='LR')
 |  
 |  An adjusted Sudoku-Solver from
 |  https://github.com/ppmx/sudoku-solver/blob/master/sudoku-z3.py
 |  sudoku-z3.py could run in a Terminal or python-notebook
 |  
 |  THIs class has a second parameter 'diagonalen' as default
 |  to force 'both' diagonals to be distinct too!
 |  
 |  you may use '' or 'R' or 'L' for three different approaches:
 |  diagonalen = '' for a normal Sudoku
 |  diagonalen = 'L' for a Sudoku with Left to right diagonal 1..9
 |  diagonalen = 'R' for a Sudoku with Right to left diagonal 1..9
 |  
 |  Methods defined here:
 |  
 |  __init__(self, inputstr='..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7..', diagonalen='LR')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  parse_grid(self)
 |

In [6]:
ST.run_z3()

count =  1
[9, 3, 1, 8, 6, 7, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[8, 4, 7, 2, 5, 1, 6, 9, 3]
[7, 8, 4, 5, 2, 9, 3, 6, 1]
[6, 9, 2, 1, 3, 8, 5, 7, 4]
[3, 1, 5, 4, 7, 6, 9, 8, 2]
[4, 5, 8, 7, 9, 2, 1, 3, 6]
[1, 7, 3, 6, 8, 5, 2, 4, 9]
[2, 6, 9, 3, 1, 4, 7, 5, 8]


solution found


In [7]:
default_start

'..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7..'

In [8]:
ST2 = SudokuPKHG( default_diagonal_two_solutions)
                 

----------.---------.----------
| .  .  1 | .  .  . | .  .  . |
| .  .  6 | .  4  . | .  .  . |
| .  .  . | .  5  1 | .  9  3 |
----------.---------.----------
| .  .  4 | .  .  . | .  .  . |
| .  9  2 | .  .  . | 5  7  . |
| .  .  . | .  .  . | 9  .  . |
----------.---------.----------
| 4  5  . | 7  9  . | .  .  . |
| .  .  . | .  .  . | 2  .  . |
| .  .  . | .  .  . | 7  .  . |
----------.---------.----------

_________DBG L75______
 string LR

DBG LR [A1, B2, C3, D4, E5, F6, G7, H8, I9]
DBG RL [A9, B8, C7, D6, E5, F4, G3, H2, I1]


In [9]:
ST2.run_z3(10)

count =  1
[9, 3, 1, 8, 6, 7, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[8, 4, 7, 2, 5, 1, 6, 9, 3]
[7, 8, 4, 5, 2, 9, 3, 6, 1]
[6, 9, 2, 1, 3, 8, 5, 7, 4]
[3, 1, 5, 4, 7, 6, 9, 8, 2]
[4, 5, 8, 7, 9, 2, 1, 3, 6]
[1, 7, 3, 6, 8, 5, 2, 4, 9]
[2, 6, 9, 3, 1, 4, 7, 5, 8]


count =  2
[9, 3, 1, 6, 8, 7, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[8, 4, 7, 2, 5, 1, 6, 9, 3]
[7, 8, 4, 5, 2, 9, 3, 6, 1]
[6, 9, 2, 1, 3, 8, 5, 7, 4]
[3, 1, 5, 4, 7, 6, 9, 8, 2]
[4, 5, 8, 7, 9, 2, 1, 3, 6]
[1, 7, 3, 8, 6, 5, 2, 4, 9]
[2, 6, 9, 3, 1, 4, 7, 5, 8]


count = 2  
 but more solutions possible!


In [10]:
#parsegrid from WWW  help(ST.parse_grid)

In [11]:
help(ST.print_Sudoku)

Help on method print_Sudoku in module __main__:

print_Sudoku(all81) method of __main__.SudokuPKHG instance
    print a Sudoku (a list of  81 given elements, a '?')
    Example all81
    all81 ='..1........6.4........51.93..4.......92...57.......9..45.79........8.2........7..'
    OR
    all81 = list(all81) for conviniance



In [12]:
ST = SudokuPKHG(default_start, diagonalen='RL' ) 

----------.---------.----------
| .  .  1 | .  .  . | .  .  . |
| .  .  6 | .  4  . | .  .  . |
| .  .  . | .  5  1 | .  9  3 |
----------.---------.----------
| .  .  4 | .  .  . | .  .  . |
| .  9  2 | .  .  . | 5  7  . |
| .  .  . | .  .  . | 9  .  . |
----------.---------.----------
| 4  5  . | 7  9  . | .  .  . |
| .  .  . | .  8  . | 2  .  . |
| .  .  . | .  .  . | 7  .  . |
----------.---------.----------

_________DBG L75______
 string RL

DBG LR [A1, B2, C3, D4, E5, F6, G7, H8, I9]
DBG RL [A9, B8, C7, D6, E5, F4, G3, H2, I1]


In [13]:
ST.run_z3(10) #maximal 10 tries

count =  1
[9, 3, 1, 8, 6, 7, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[8, 4, 7, 2, 5, 1, 6, 9, 3]
[7, 8, 4, 5, 2, 9, 3, 6, 1]
[6, 9, 2, 1, 3, 8, 5, 7, 4]
[3, 1, 5, 4, 7, 6, 9, 8, 2]
[4, 5, 8, 7, 9, 2, 1, 3, 6]
[1, 7, 3, 6, 8, 5, 2, 4, 9]
[2, 6, 9, 3, 1, 4, 7, 5, 8]


solution found


In [14]:
ST.count

1

In [15]:
#forcing A1 .. I9 to be different
ST_R =  SudokuPKHG(diagonalen='L' )

----------.---------.----------
| .  .  1 | .  .  . | .  .  . |
| .  .  6 | .  4  . | .  .  . |
| .  .  . | .  5  1 | .  9  3 |
----------.---------.----------
| .  .  4 | .  .  . | .  .  . |
| .  9  2 | .  .  . | 5  7  . |
| .  .  . | .  .  . | 9  .  . |
----------.---------.----------
| 4  5  . | 7  9  . | .  .  . |
| .  .  . | .  8  . | 2  .  . |
| .  .  . | .  .  . | 7  .  . |
----------.---------.----------

_________DBG L75______
 string L

DBG LR [A1, B2, C3, D4, E5, F6, G7, H8, I9]


In [16]:
ST_R.run_z3(10)

count =  1
[9, 3, 1, 6, 7, 8, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[7, 4, 8, 2, 5, 1, 6, 9, 3]
[1, 7, 4, 5, 2, 9, 3, 8, 6]
[3, 9, 2, 8, 6, 4, 5, 7, 1]
[8, 6, 5, 1, 3, 7, 9, 4, 2]
[4, 5, 3, 7, 9, 2, 1, 6, 8]
[6, 1, 7, 4, 8, 5, 2, 3, 9]
[2, 8, 9, 3, 1, 6, 7, 5, 4]


count =  2
[9, 3, 1, 6, 7, 8, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[7, 4, 8, 2, 5, 1, 6, 9, 3]
[1, 7, 4, 5, 2, 9, 3, 8, 6]
[3, 9, 2, 8, 6, 4, 5, 7, 1]
[8, 6, 5, 3, 1, 7, 9, 4, 2]
[4, 5, 3, 7, 9, 2, 1, 6, 8]
[6, 1, 7, 4, 8, 5, 2, 3, 9]
[2, 8, 9, 1, 3, 6, 7, 5, 4]


count =  3
[9, 3, 1, 6, 7, 8, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[7, 4, 8, 2, 5, 1, 6, 9, 3]
[1, 7, 4, 5, 2, 9, 3, 8, 6]
[8, 9, 2, 3, 6, 4, 5, 7, 1]
[3, 6, 5, 8, 1, 7, 9, 4, 2]
[4, 5, 3, 7, 9, 2, 1, 6, 8]
[6, 1, 7, 4, 8, 5, 2, 3, 9]
[2, 8, 9, 1, 3, 6, 7, 5, 4]


count =  4
[9, 3, 1, 6, 7, 8, 4, 2, 5]
[5, 2, 6, 9, 4, 3, 8, 1, 7]
[7, 4, 8, 2, 5, 1, 6, 9, 3]
[1, 7, 4, 5, 2, 9, 3, 6, 8]
[8, 9, 2, 3, 6, 4, 5, 7, 1]
[3, 6, 5, 8, 1, 7, 9, 4, 2]
[4, 5, 3, 7, 9, 2, 1, 8, 6

In [17]:
ST_R.count

10

# if there are differnent solutions use copy past ...
see next two cells

In [18]:
count1 = [
[5, 4, 1, 9, 3, 8, 6, 2, 7],
[9, 3, 6, 2, 4, 7, 1, 5, 8],
[7, 2, 8, 6, 5, 1, 4, 9, 3],
[1, 7, 4, 5, 2, 9, 3, 8, 6],
[8, 9, 2, 1, 6, 3, 5, 7, 4],
[3, 6, 5, 8, 7, 4, 9, 1, 2],
[4, 5, 3, 7, 9, 2, 8, 6, 1],
[6, 1, 7, 3, 8, 5, 2, 4, 9],
[2, 8, 9, 4, 1, 6, 7, 3, 5]]

#=============

count2 = [
[9, 3, 1, 8, 7, 6, 4, 2, 5],
[5, 2, 6, 9, 4, 3, 8, 1, 7],
[8, 4, 7, 2, 5, 1, 6, 9, 3],
[7, 8, 4, 5, 2, 9, 1, 3, 6],
[1, 9, 2, 6, 3, 8, 5, 7, 4],
[3, 6, 5, 4, 1, 7, 9, 8, 2],
[4, 5, 8, 7, 9, 2, 3, 6, 1],
[6, 7, 3, 1, 8, 4, 2, 5, 9],
[2, 1, 9, 3, 6, 5, 7, 4, 8]]


In [19]:
for i in range(9):
    for j in range(9):
        c1 = count1[i][j]
        c2 = count2[i][j]
        if c1 != c2:
            print('{} {} c1 {} c2 {}'.format(i,j,c1,c2))

0 0 c1 5 c2 9
0 1 c1 4 c2 3
0 3 c1 9 c2 8
0 4 c1 3 c2 7
0 5 c1 8 c2 6
0 6 c1 6 c2 4
0 8 c1 7 c2 5
1 0 c1 9 c2 5
1 1 c1 3 c2 2
1 3 c1 2 c2 9
1 5 c1 7 c2 3
1 6 c1 1 c2 8
1 7 c1 5 c2 1
1 8 c1 8 c2 7
2 0 c1 7 c2 8
2 1 c1 2 c2 4
2 2 c1 8 c2 7
2 3 c1 6 c2 2
2 6 c1 4 c2 6
3 0 c1 1 c2 7
3 1 c1 7 c2 8
3 6 c1 3 c2 1
3 7 c1 8 c2 3
4 0 c1 8 c2 1
4 3 c1 1 c2 6
4 4 c1 6 c2 3
4 5 c1 3 c2 8
5 3 c1 8 c2 4
5 4 c1 7 c2 1
5 5 c1 4 c2 7
5 7 c1 1 c2 8
6 2 c1 3 c2 8
6 6 c1 8 c2 3
7 1 c1 1 c2 7
7 2 c1 7 c2 3
7 3 c1 3 c2 1
7 5 c1 5 c2 4
7 7 c1 4 c2 5
8 1 c1 8 c2 1
8 3 c1 4 c2 3
8 4 c1 1 c2 6
8 5 c1 6 c2 5
8 7 c1 3 c2 4
8 8 c1 5 c2 8


In [20]:
'the END or use numpy see cells 21 ... '

'the END or use numpy see cells 21 ... '

In [21]:
import numpy as np

In [22]:
npa1 = np.array(count1)
npa2 = np.array(count2)

In [23]:
npa1 == npa2

array([[False, False,  True, False, False, False, False,  True, False],
       [False, False,  True, False,  True, False, False, False, False],
       [False, False, False, False,  True,  True, False,  True,  True],
       [False, False,  True,  True,  True,  True, False, False,  True],
       [False,  True,  True, False, False, False,  True,  True,  True],
       [ True,  True,  True, False, False, False,  True, False,  True],
       [ True,  True, False,  True,  True,  True, False,  True,  True],
       [ True, False, False, False,  True, False,  True, False,  True],
       [ True, False,  True, False, False, False,  True, False, False]])

In [24]:
npa1 - npa2 # 0 zero means: same value ;-)

array([[-4,  1,  0,  1, -4,  2,  2,  0,  2],
       [ 4,  1,  0, -7,  0,  4, -7,  4,  1],
       [-1, -2,  1,  4,  0,  0, -2,  0,  0],
       [-6, -1,  0,  0,  0,  0,  2,  5,  0],
       [ 7,  0,  0, -5,  3, -5,  0,  0,  0],
       [ 0,  0,  0,  4,  6, -3,  0, -7,  0],
       [ 0,  0, -5,  0,  0,  0,  5,  0,  0],
       [ 0, -6,  4,  2,  0,  1,  0, -1,  0],
       [ 0,  7,  0,  1, -5,  1,  0, -1, -3]])